<a href="https://colab.research.google.com/github/vijaykeshri/Learning/blob/develop/fine-tune/Mistral7bMetQLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***fine-tuning*** takes much less data and computation compared to the previous pre-trained/foundation model but it’s still not possible on consumer-grade hardware.

**PEFT(parameter efficient fine-tuning)** fixes this issue and applies clever methods to make fine-tuning possible even on the free tier of Google Colab.

Lets use **Quantization** and **LoRA**(Low-Rank Adaptation) to fine-tune Mistral 7b instruct and introduce new knowledge to it.

**LoRA** freezes the model's parameters(W0), attaches small *trainable adapter layers(ΔW = BA) to it, and only trains the adapter*. this reduces the trainable parameters by a lot and consumes much less RAM. One important hyperparameter in LoRA is the r, in this example r=2.

# **Data Preparation:**
In this example, we are going to fine-tune Mistral on the Enlighten’s codebase which is just a bunch of C# classes. Keep in mind, these raw C# classes can be used to fine-tune base non-instruct models but the resulting model will also be non-instruct and will just complete a document, mostly used for code completion.

> But what we need:


---


we need **instruction-based data in the form of question/answer pairs** so that our model can work/aswer on questions ask about the classes/methods etc. used in codebase.


> What we should do:


To acheive this, we are going to use **a much larger and stronger model to generate our data** based on the code base *synthetically*. Author chose the newly released Google Gemini Pro for this task since it is both free for this use-case and powerful. (Gpt-4 would be the best model but the API is not free).



> Training and Testing dataset

We need two datasets, one for training and one for testing, **both will be synthetically generated by Gemini**. Training data will be a question about the code and its answer. The testing dataset will be in the form of multiple choice questions, it is a question followed by 4 choices and the correct one.

For each C# class, Author generated **20 Q/A** for training and **3 **for testing. 20 training questions were **10 just-code questions** and **10 general questions**. There were about 90 C# classes in total.

Now we are going **to give each class to Gemini and ask it to generate our data, for this, we need our own customized system message**. These are the System messages *Author* engineered.





# Ststem Message Strategy:
1. Tell What to Do or What's purpose
2. Rules/Instrunctions to be followed
3. Response format

# System message for 10 just-code questions (Training Data)

> Tell what to do:

 you take a C# class from a Unity project with it's documentaion and create 10 question/answer pairs for it.

> Rules/Instrunctions to be followed:

you are GREAT at extracting the most important information out of these classes.
all questions should ask for code and all answers should be C# code.
questions should have context.
DO NOT say 'the code' or 'this code' in your questions and do not refer to it at all.
mention the class name in all questions.

> Response Format and its instrunction/rules:

your answer MUST be a valid json format, a list of 10 objects, each object has 3 text fields: class name, Q(question), A(answer)
any '\n' in the text fields MUST be '\\n' so that when reading it later on, we won't run into any issues.





In [ ]:
import pandas as pd